In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import copy
import pandas as pd
import time
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from utils.subspace_clustering_helper_funcs import *
from utils.preprocessing import *

In [2]:
print("Loading")

# Kai's laptop
data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\saved_datasets\\"
model_dir_path = 'C:\\Users\\kdmen\\Desktop\\Research\\Repos\\fl-gestures\\models\\Embedding\\Autoencoders\\'
metadata_cols_df = pd.read_pickle('C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\saved_datasets\\metadata_cols_df.pkl')
# BRC Desktop
#data_path = "D:\\Kai_MetaGestureClustering_24\\saved_datasets\\"
#model_dir_path = 'C:\\Users\\YamagamiLab\\Desktop\\Dev\\fl-gestures\\models\\Embedding\\Autoencoders\\'
#metadata_cols_df = pd.read_pickle('D:\\Kai_MetaGestureClustering_24\\saved_datasets\\metadata_cols_df.pkl')

emg_dir = "filtered_datasets\\EMG_PPD\\"
both_dir = "filtered_datasets\\Both_PPD\\"

metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']

emg_training_users_df = pd.read_pickle(data_path+emg_dir+'training_users_df.pkl').drop(metadata_cols, axis=1)
emg_test_users_df = pd.read_pickle(data_path+emg_dir+'test_users_df.pkl').drop(metadata_cols, axis=1)

both_training_users_df = pd.read_pickle(data_path+both_dir+'training_users_df.pkl').drop(metadata_cols, axis=1)
both_test_users_df = pd.read_pickle(data_path+both_dir+'test_users_df.pkl').drop(metadata_cols, axis=1)

full_emg_users_df = pd.concat([emg_training_users_df, emg_test_users_df])
full_both_users_df = pd.concat([both_training_users_df, both_test_users_df])


Loading


In [3]:
data_df = pd.concat([metadata_cols_df, full_both_users_df], axis=1)

print(data_df.shape)
data_df.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


## Filter out P111 P124 and P131 and unimpared participants

In [4]:
filtered_df = data_df[~data_df['Participant'].isin(['P004', 'P005', 'P006', 'P008', 'P010', 'P011', 'P111','P124','P131'])]
print(filtered_df.shape)
filtered_df.head()

(147200, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [5]:
print(len(filtered_df['Participant'].unique()))

23


## Standardization
> Kai's loaded in training/testing datasets are already standardized... EMG is negative so

In [6]:
#IMU_df = filtered_df.iloc[:, 3:75]
#EMG_df = filtered_df.iloc[:, 75:91]
#IMU_mean = IMU_df.values.mean()
#EMG_mean = EMG_df.values.mean()
#IMU_sd = IMU_df.values.std()
#EMG_sd = EMG_df.values.std()
#IMU_scaled = pd.DataFrame((IMU_df - IMU_mean) / IMU_sd)
#EMG_scaled = pd.DataFrame((EMG_df - EMG_mean) / EMG_sd)
#scaled_signal_df = pd.concat([IMU_scaled, EMG_scaled], axis=1)
#print(scaled_signal_df.shape)
#scaled_signal_df.head()

In [7]:
#preprocess_df_by_gesture(data_df, preprocessing_approach, biosignal_switch_ix_lst=[72], trial_length=64)

scaled_signal_df = filtered_df.iloc[:, 3:]

print(scaled_signal_df.shape)
scaled_signal_df.head()

(147200, 88)


,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,IMU2_ay,IMU2_az,IMU2_vx,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,-0.883268,-0.327978,-0.221822,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,-0.944195,-0.363602,-0.149833,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,-0.944029,-0.357476,-0.164447,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,-0.911993,-0.335594,-0.006214,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,-0.939228,-0.357917,0.009543,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


## Perform PCA

In [8]:
pca = PCA(n_components=3)
#pca_by_signal_result_df = pd.DataFrame(data=pca.fit_transform(scaled_signal_df), columns=['PC1', 'PC2', 'PC3'])
#pca_by_column_result_df = pd.DataFrame(data=pca.fit_transform(scaled_column_df), columns=['PC1', 'PC2', 'PC3'])
pca_df = pd.DataFrame(data=pca.fit_transform(scaled_signal_df), columns=['PC1', 'PC2', 'PC3']).reset_index(drop=True)
filtered_df = filtered_df.reset_index(drop=True)
#pca_by_signal_result_df = pca_by_signal_result_df.reset_index(drop=True)
#pca_by_column_result_df = pca_by_column_result_df.reset_index(drop=True)
#pca_by_signal_df = pd.concat([filtered_df.iloc[:,0:3], pca_by_signal_result_df], axis=1)
#pca_by_column_df = pd.concat([filtered_df.iloc[:,0:3], pca_by_column_result_df], axis=1)
pca_df = pd.concat([filtered_df.iloc[:,:3], pca_df], axis=1)

print(pca_df.shape)
pca_df.head()


(147200, 6)


,Participant,Gesture_ID,Gesture_Num,PC1,PC2,PC3
0,P102,pan,1,-0.001759,0.046219,0.152081
1,P102,pan,1,-0.092202,0.103943,0.149814
2,P102,pan,1,-0.340428,0.298297,0.315578
3,P102,pan,1,0.024455,0.252892,0.307412
4,P102,pan,1,-0.569416,0.314727,0.418419


## Plot PCA

In [ ]:
# Create the 2D scatter plot
fig = px.scatter(pca_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Participant', 
                 title='2D PCA of Gesture Data Standardized by Channel, Coloring Based on Participant',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
# Create the 2D scatter plot
fig = px.scatter(pca_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Gesture_ID', 
                 title='2D PCA of Gesture Data Standardized by Channel, Coloring Based on Gesture',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
# Convert Participant and Gesture_ID to categorical codes for color and symbol
# Create the 2D scatter plot
fig = px.scatter(pca_by_signal_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Participant', 
                 title='2D PCA of Gesture Data Standardized by Signal, Coloring Based on Participant',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [ ]:
# Convert Participant and Gesture_ID to categorical codes for color and symbol
# Create the 2D scatter plot
fig = px.scatter(pca_df, 
                 x='PC1', 
                 y='PC2', 
                 color='Gesture_ID', 
                 title='2D PCA of Gesture Data Standardized by Signal, Coloring Based on Gesture',
                 )

# Update layout to remove axis numbers and increase plot size
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    xaxis=dict(showticklabels=False, title='PC1'),
    yaxis=dict(showticklabels=False, title='PC2'),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.show()

In [13]:
fig = px.scatter_3d(pca_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Gesture_ID', 
                    title='3D PCA of Biosignal Data, Standardized by Channel, Coloring Based on Gesture')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

NameError: name 'pca_by_column_df' is not defined

In [ ]:
fig = px.scatter_3d(pca_by_column_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Participant', 
                    title='3D PCA of Biosignal Data, Standardized by Channel, Coloring Based on Participant')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Gesture_ID', 
                    title='3D PCA of Biosignal Data, Standardized by Signal, Coloring Based on Gesture')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_by_signal_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Participant', 
                    title='3D PCA of Biosignal Data, Standardized by Signal, Coloring Based on Participant')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Add Handedness 

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Read all sheets into a dictionary of dataframes
all_sheets_dict = pd.read_excel("C:\\Users\\rubin\\Research\\user_gesture_descriptions_qualitative_analysis.xlsx", sheet_name=None)

# Initialize an empty list to hold individual dataframes
df_list = []

# Iterate through the dictionary and process each sheet
for user_id, df in all_sheets_dict.items():
    df.columns.values[1] = 'gesture'
    # Identify columns with "Unnamed" in their names
    unnamed_cols = df.columns.str.contains("Unnamed")
    # Drop those columns
    df = df.loc[:, ~unnamed_cols]
    df = df.iloc[:10,:]
    df= df.reset_index(drop=True)
    df_list.append(df)  # Add the dataframe to the list

In [18]:
print(df_list[2].shape)
df_list[2].head()

(10, 17)


,PID,gesture,time,description,hand used,lifted arms off arm rest?,dynamic gesture?,muscles activated 1,muscles activated 2,muscles activated 3,agreement,midair or on table,gesture ideas origin 1,gesture ideas origin 2,gesture ideas origin 3,one hand v 2 hands,one hand v 2 hands.1
0,P103,rotate,3,make a circle with right hand without moving f...,right,barely,yes,upper-body (bicep/tricep),shoulder,NaN,rotate from shoulders,midair,what I can do with my abilities,NaN,NaN,due to handedness,NaN
1,P103,move,2,move right hand from left to right without rai...,right,barely,yes,upper-body (bicep/tricep),NaN,NaN,point and move entire hand in 2D (like on a fl...,on a 2D plane but not on table,NaN,NaN,NaN,NaN,NaN
2,P103,select-single,2,using a mouse to select,right,barely,yes,shoulder,upper-body (bicep/tricep),NaN,point at table (like clicking a mouse),on a 2D plane but not on table,NaN,NaN,NaN,NaN,NaN
3,P103,zoom-in,3,move head forward and backwards,head,no,yes,shoulder,NaN,NaN,move head closer/away to screen,neither,NaN,NaN,NaN,NaN,NaN
4,P103,zoom-out,3,"move head backwards, then forward",head,no,yes,shoulder,NaN,NaN,move head closer/away to screen,neither,NaN,NaN,NaN,NaN,NaN


In [19]:
# Concatenate all DataFrames in the list into one large DataFrame
all_PID_xlsx_df = pd.concat(df_list[1:], ignore_index=True)
# Optionally, reset the index if needed
all_PID_xlsx_df.reset_index(drop=True, inplace=True)
all_PID_xlsx_df.rename(columns={'PID': 'Participant', 'gesture': 'Gesture_ID', 'hand used': 'Handedness', 'lifted arms off arm rest?': 'Lifted Arms' }, inplace=True)
print(all_PID_xlsx_df.shape)
all_PID_xlsx_df.head()

(270, 22)


,Participant,Gesture_ID,time,description,Handedness,Lifted Arms,dynamic gesture?,muscles activated 1,muscles activated 2,muscles activated 3,...,gesture ideas origin 1,gesture ideas origin 2,gesture ideas origin 3,one hand v 2 hands,one hand v 2 hands.1,notes,muscles activated 4,body part used 1,body part used 2,body part used 3
0,P102,rotate,4,rotate entire right hand from wrist,right,yes,yes,wrist/forearm,shoulder,fingers,...,tablet/touchscreen,NaN,NaN,due to handedness,other hand is too disabled,NaN,NaN,NaN,NaN,NaN
1,P102,move,2,point right hand and move entire arm from left...,right,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P102,select-single,3,point at object using right index finger,right,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P102,zoom-in,2,use two hands to make a frame and move hands c...,both,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P102,zoom-out,5,use right thumb and index finger and pinch fin...,right,yes,yes,fingers,wrist/forearm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
filtered_metadata_df = all_PID_xlsx_df.loc[:,['Participant','Gesture_ID','Handedness', 'Lifted Arms','muscles activated 1']]
print(filtered_metadata_df.shape)
# make sure same users are in both dfs
filtered_metadata_df = filtered_metadata_df[filtered_metadata_df['Participant'].isin(['P102','P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P110', 'P112', 'P114', 'P115', 'P116', 'P118', 'P119', 'P121', 'P122', 'P123', 'P125', 'P126',
 'P127', 'P128', 'P132'])]
print(filtered_metadata_df.shape)
print(set(pca_by_signal_df['Participant'].unique()) == set(filtered_metadata_df['Participant'].unique()))

(270, 5)
(230, 5)
True


In [21]:
pca_signal_meta_df = pd.merge(filtered_metadata_df, pca_by_signal_df, on=['Participant', 'Gesture_ID'], how='inner')
pca_column_meta_df = pd.merge(filtered_metadata_df, pca_by_column_df, on=['Participant', 'Gesture_ID'], how='inner')
print(pca_signal_meta_df.shape)
pca_signal_meta_df.head()


(147200, 9)


,Participant,Gesture_ID,Handedness,Lifted Arms,muscles activated 1,Gesture_Num,PC1,PC2,PC3
0,P102,rotate,right,yes,wrist/forearm,1,-0.239026,-0.611281,-0.876869
1,P102,rotate,right,yes,wrist/forearm,1,-0.249440,-0.677231,-0.860271
2,P102,rotate,right,yes,wrist/forearm,1,-0.149920,-0.656097,-0.852871
3,P102,rotate,right,yes,wrist/forearm,1,-0.202984,-0.613159,-0.785642
4,P102,rotate,right,yes,wrist/forearm,1,0.551187,-0.709154,-0.579720


## Make sure labeling is consistent

In [23]:
print(pca_signal_meta_df['Handedness'].unique())
print(pca_signal_meta_df['Lifted Arms'].unique())
print(pca_signal_meta_df['muscles activated 1'].unique())

['right' 'both' 'head' 'left' 'shoulder']
['yes' 'barely' 'no']
['wrist/forearm' 'shoulder' 'fingers' 'upper-body (bicep/tricep)']


## plot based on handedness

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Handedness',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Handedness')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Handedness'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Handedness',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Handedness')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Handedness'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## plot Based on Lifted Arms

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Lifted Arms',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Lifting Arms')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Lifted Arms'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Lifted Arms',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Lifting Arms')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Lifted Arms'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Plot based on primary muscles activated

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='muscles activated 1',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Primary Muscles Activated')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['muscles activated 1'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        title=dict(text='Primary Muscles Activated'),
        x=0.7,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='muscles activated 1',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Primary Muscles Activated')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['muscles activated 1'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        title=dict(text='Primary Muscles Activated'),
        x=0.7,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()